# (분석 전용 기능 정의)

In [8]:
import json
import pprint

class IndyPrinter:
    
    line_char='-'
    line_length=100
    ref_line_char='#'
    ref_line_length=50
    
    class __JsonPrintType:
        NOT_JSON = 0
        PURE_JSON = 1
        MULTIPLE_JSON_STR = 2
        
    def __init__(self, data):
        self.__result, self.__is_json = self.__type_filter(data)
        
    def __call__(self, print_title=None, *, use_pp=True):
        pp = pprint.PrettyPrinter(width=40, indent=2, compact=True)
        
        self.__line(self.line_char, self.line_length)
        if print_title is not None : print(print_title.center(self.line_length, self.line_char))
        if not self.__is_json : print(self.__result)
        elif use_pp : pp.pprint(self.__result)
        else : self.__json_print()
        self.__line(self.line_char, self.line_length)
    
    def default_mode(self):
        self.line_char='-'
        self.line_length=100
        self.ref_line_char='^'
        self.ref_line_length=50
        
    def __type_filter(self, data):
        _data = data
        data_type = type(_data)
        is_json = True
        
        try:
            if data_type == 'dict' : 
                _data = json.dumps(_data)
            _data = json.loads(_data)
        except json.JSONDecodeError:
            is_json = False
        finally:
            if(is_json) : return (_data, is_json)
            return (_data, is_json)
    
    def __line(self, char, length):
        print(char * length)
    
    def __json_print(self):
        json_obj = self.__result
        print(f'<json_keys> : {[key for key in json_obj]}')
        spec = self.__analyze_structure(json_obj)
        json_info, references = self.__mapping(json_obj, spec)
        print(json.dumps(json_info, indent=4))
        self.__line(self.ref_line_char, self.ref_line_length)
        print("REFERENCES".center(self.ref_line_length, self.ref_line_char))
        for ref_name in references: 
            print(f'{ref_name} : {json.dumps(references[ref_name], indent=4)}')
        self.__line(self.ref_line_char, self.ref_line_length)
    
    def __analyze_structure(self, json_obj):
        result = []
        for k, v in data.items():
            parse_type = None
            if type(v) != int and v[0] == '{' and v[-1] == '}' : 
                parse_type = self.__JsonPrintType.PURE_JSON
                if '\n' in v : 
                    parse_type = self.__JsonPrintType.MULTIPLE_JSON_STR
            else: 
                parse_type = self.__JsonPrintType.NOT_JSON
            result.append((k, parse_type))
        return dict(result)
    
    def __mapping(self, json_obj, spec):
        
        details = dict()
        
        for key in json_obj:
            if spec[key] == self.__JsonPrintType.MULTIPLE_JSON_STR:
                name = '[[{}]]'.format(key)
                details[name] = list(map(lambda json_str: json.loads(json_str), json_obj[key].split('\n')))
                json_obj[key] = f'{name} : please ignore list'
            elif spec[key] == self.__JsonPrintType.PURE_JSON:
                json_obj[key] = json.loads(json_obj[key])
            elif spec[key] == self.__JsonPrintType.NOT_JSON:
                pass
            
        return json_obj, details
    
class PrintFuncIO:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print('[input] tuple params : ', end='')
        pprint.pprint(args)
        print('[input]dict params : ', end='')
        pprint.pprint(kwargs)
        print('[output]result : ', end='')
        result = self.func(*args, **kwargs)
        pprint.pprint(result)
        return result

# 1. 필요한 모듈 불러오기

In [9]:
# Core Modules
from indy import anoncreds, did, ledger, pool, wallet, blob_storage
from indy.error import ErrorCode, IndyError
import asyncio
import time

# Utils
import json
from os import environ
from pathlib import Path
from tempfile import gettempdir
import sys
from ctypes import *
from os.path import dirname

<br><br><br><hr>

# 2. 유틸함수 및 주요 변수 선언

In [10]:
PROTOCOL_VERSION = 2

def default_path_home() -> Path:
    """
    @return : .indy_client path home 
    
    [Description]:
    indy 및 aries의 default 경로를 반환한다.
    """
    return Path.home().joinpath(".indy_client")

def tmp_path_home() -> Path:
    """
    @return : 임시폴더안의 indy 경로 ( if linux /tmp/indy )
    
    [Description]:
    테스트를 위한 임시폴더 경로를 만들어 반환한다.
    """
    return Path(gettempdir()).joinpath("indy")

def local_indy_pool_genesis_txn_data(pool_ip : str = "127.0.0.1") -> str:
    """
    @params pool_ip : pool ledger에 저장될 참여 노드 ip 주소
    @return : genesis txn data
    
    [Description]:
    indy-SDK SAMPLE 에서 제공하는 indy-node에서 정해진 indy_pool 도커 컨테이너에 접근하는 Genesis 데이터를 반환함
    """
    #pool_ip = environ.get("TEST_POOL_IP", "127.0.0.1")

    return "\n".join([
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"{}","client_port":9702,"node_ip":"{}","node_port":9701,"services":["VALIDATOR"]}},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"}},"metadata":{{"from":"Th7MpTaRZVRYnPiabds81Y"}},"type":"0"}},"txnMetadata":{{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG7zqS8TK1MXwuCHj1FKNzVpsnafmqLG1vXN88rt38mNFs9TENzm4QHdBzsvCuoBnPH7rpYYDo9DZNJePaDvRvqJKByCabubJz3XXKbEeshzpz4Ma5QYpJqjk","blskey_pop":"Qr658mWZ2YC8JXGXwMDQTzuZCWF7NK9EwxphGmcBvCh6ybUuLxbG65nsX4JvD4SPNtkJ2w9ug1yLTj6fgmuDg41TgECXjLCij3RMsV8CwewBVgVN67wsA45DFWvqvLtu4rjNnE9JbdFTc1Z4WCPA3Xan44K1HoHAq9EVeaRYs8zoF5","client_ip":"{}","client_port":9704,"node_ip":"{}","node_port":9703,"services":["VALIDATOR"]}},"dest":"8ECVSk179mjsjKRLWiQtssMLgp6EPhWXtaYyStWPSGAb"}},"metadata":{{"from":"EbP4aYNeTHL6q385GuVpRV"}},"type":"0"}},"txnMetadata":{{"seqNo":2,"txnId":"1ac8aece2a18ced660fef8694b61aac3af08ba875ce3026a160acbc3a3af35fc"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node3","blskey":"3WFpdbg7C5cnLYZwFZevJqhubkFALBfCBBok15GdrKMUhUjGsk3jV6QKj6MZgEubF7oqCafxNdkm7eswgA4sdKTRc82tLGzZBd6vNqU8dupzup6uYUf32KTHTPQbuUM8Yk4QFXjEf2Usu2TJcNkdgpyeUSX42u5LqdDDpNSWUK5deC5","blskey_pop":"QwDeb2CkNSx6r8QC8vGQK3GRv7Yndn84TGNijX8YXHPiagXajyfTjoR87rXUu4G4QLk2cF8NNyqWiYMus1623dELWwx57rLCFqGh7N4ZRbGDRP4fnVcaKg1BcUxQ866Ven4gw8y4N56S5HzxXNBZtLYmhGHvDtk6PFkFwCvxYrNYjh","client_ip":"{}","client_port":9706,"node_ip":"{}","node_port":9705,"services":["VALIDATOR"]}},"dest":"DKVxG2fXXTU8yT5N7hGEbXB3dfdAnYv1JczDUHpmDxya"}},"metadata":{{"from":"4cU41vWW82ArfxJxHkzXPG"}},"type":"0"}},"txnMetadata":{{"seqNo":3,"txnId":"7e9f355dffa78ed24668f0e0e369fd8c224076571c51e2ea8be5f26479edebe4"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node4","blskey":"2zN3bHM1m4rLz54MJHYSwvqzPchYp8jkHswveCLAEJVcX6Mm1wHQD1SkPYMzUDTZvWvhuE6VNAkK3KxVeEmsanSmvjVkReDeBEMxeDaayjcZjFGPydyey1qxBHmTvAnBKoPydvuTAqx5f7YNNRAdeLmUi99gERUU7TD8KfAa6MpQ9bw","blskey_pop":"RPLagxaR5xdimFzwmzYnz4ZhWtYQEj8iR5ZU53T2gitPCyCHQneUn2Huc4oeLd2B2HzkGnjAff4hWTJT6C7qHYB1Mv2wU5iHHGFWkhnTX9WsEAbunJCV2qcaXScKj4tTfvdDKfLiVuU2av6hbsMztirRze7LvYBkRHV3tGwyCptsrP","client_ip":"{}","client_port":9708,"node_ip":"{}","node_port":9707,"services":["VALIDATOR"]}},"dest":"4PS3EDQ3dW1tci1Bp6543CfuuebjFrg36kLAUcskGfaA"}},"metadata":{{"from":"TWwCRQRZ2ZHMJFn9TzLp7W"}},"type":"0"}},"txnMetadata":{{"seqNo":4,"txnId":"aa5e817d7cc626170eca175822029339a444eb0ee8f0bd20d3b0b76e566fb008"}},"ver":"1"}}'.format(
            pool_ip, pool_ip)
    ])

def save_pool_genesis_txn_file(path : str, genesis_txn_data : str) -> None:
    """
    @params path : genesis txn 데이터를 저장할 위치
    @params genesis_txn_data : genesis txn 데이터
    
    [Description]:
    genesis txn 데이터를 파일로 저장
    """
    data = genesis_txn_data

    path.parent.mkdir(parents=True, exist_ok=True)

    with open(str(path), "w+") as f:
        f.writelines(data)
        
async def ensure_previous_request_applied(pool_handle, checker_request, checker):
    for _ in range(3):
        response = json.loads(await ledger.submit_request(pool_handle, checker_request))
        IndyPrinter(response)("ensure_previous_request_applied-resposne", use_pp = True) #DEBUG
        try:
            if checker(response):
                print("return!!!")
                return json.dumps(response)
        except TypeError:
            pass

<br><br><br><hr>

# 3. Make Pool Ledger Handler

In [11]:
local_indy_pool_genesis_txn_data("10.0.2.15")

'{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"10.0.2.15","client_port":9702,"node_ip":"10.0.2.15","node_port":9701,"services":["VALIDATOR"]},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"},"metadata":{"from":"Th7MpTaRZVRYnPiabds81Y"},"type":"0"},"txnMetadata":{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"},"ver":"1"}\n{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG7zqS8TK1MXwuCHj1FKNzVpsnafmqLG1vXN88rt38mNFs9TENzm4QHdBzsvCuoBnPH7rpYYDo9DZNJePaDvRvqJKByCabubJz3XXKbEeshzp

In [15]:
pool_ = dict()

pool_['name'] = 'pool3'
genesis_txn_save_path = tmp_path_home().joinpath("{}.txn".format(pool_['name']))

pool_['genesis_txn_path'] = genesis_txn_save_path

pool_['config'] = json.dumps({"genesis_txn": str(pool_['genesis_txn_path'])}) # Path -> str

pool_['genesis_txn_data'] = local_indy_pool_genesis_txn_data("10.0.2.15")

save_pool_genesis_txn_file(pool_['genesis_txn_path'], pool_['genesis_txn_data'])

await pool.set_protocol_version(PROTOCOL_VERSION)

try:
    await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
except IndyError as ex:
    if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
        pass

pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)
print_result = IndyPrinter(pool_)

In [16]:
print_result("create_pool_and_open_pool", use_pp = True)

----------------------------------------------------------------------------------------------------
-------------------------------------create_pool_and_open_pool--------------------------------------
{ 'config': '{"genesis_txn": '
            '"/tmp/indy/pool3.txn"}',
  'genesis_txn_data': '{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"10.0.2.15","client_port":9702,"node_ip":"10.0.2.15","node_port":9701,"services":["VALIDATOR"]},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"},"metadata":{"from":"Th7MpTaRZVRYnPiabds81Y"},"type":"0"},"txnMetadata":{"seqNo":1,"txnId":"fea82e10e894419fe

<br><br><br><hr>

# 4. Set Steward Information and Create Wallet

## - Wallet 관련 함수 정의

In [17]:
async def create_wallet(identity) -> int:
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'],identity['wallet_credentials'])

## - 지갑 생성

In [18]:
steward = {
    'name': "Sovrin Steward",
    'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
    'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
    'pool': pool_['handle'],
    'seed': '000000000000000000000000Steward1'
}

await create_wallet(steward)

steward['did_info'] = json.dumps({'seed': steward['seed']})
steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])
print_result2 = IndyPrinter(steward)

In [19]:
print_result2("steward", use_pp = True)

----------------------------------------------------------------------------------------------------
----------------------------------------------steward-----------------------------------------------
{ 'did': 'Th7MpTaRZVRYnPiabds81Y',
  'did_info': '{"seed": '
              '"000000000000000000000000Steward1"}',
  'key': 'FYmoFw55GeQH7SRFa37dkx1d2dZ3zUF8ckg7wmL7ofN4',
  'name': 'Sovrin Steward',
  'pool': 10,
  'seed': '000000000000000000000000Steward1',
  'wallet': 13,
  'wallet_config': '{"id": '
                   '"sovrin_steward_wallet"}',
  'wallet_credentials': '{"key": '
                        '"steward_wallet_key"}'}
----------------------------------------------------------------------------------------------------


<br><br><br><hr>

# 5. Getting Endorser Credentials

## - Verinym 관련 함수 정의

In [20]:
async def getting_verinym(from_ : dict, to : dict):
    """
    
    """
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    """
    
    """
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)

### (1) Government getting verinym

In [21]:
government = {
    'name': 'Government',
    'wallet_config': json.dumps({'id': 'government_wallet'}),
    'wallet_credentials': json.dumps({'key': 'government_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, government)
print_result3 = IndyPrinter(government)

In [22]:
print_result3("government", use_pp = True)

----------------------------------------------------------------------------------------------------
---------------------------------------------government---------------------------------------------
{ 'did': 'DP92P2qRue3B7QaaM3ajWz',
  'key': '7kQmatTbAxGbEHn91Y8U7jsHLuznsvTt5PDgmxhhECpp',
  'name': 'Government',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'wallet': 15,
  'wallet_config': '{"id": '
                   '"government_wallet"}',
  'wallet_credentials': '{"key": '
                        '"government_wallet_key"}'}
----------------------------------------------------------------------------------------------------


### (2) Faber(university) getting verinym

In [23]:
faber = {
    'name': 'Faber',
    'wallet_config': json.dumps({'id': 'faber_wallet'}),
    'wallet_credentials': json.dumps({'key': 'faber_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, faber)
print_result4 = IndyPrinter(faber)

In [24]:
print_result4("faber", use_pp = True)

----------------------------------------------------------------------------------------------------
-----------------------------------------------faber------------------------------------------------
{ 'did': 'FyckJRmcaRJYeHukogSij5',
  'key': '9ARg7ByY8TUaYufZ1ezZwe7ArET1X3vsXm43NDbyda3y',
  'name': 'Faber',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'wallet': 19,
  'wallet_config': '{"id": '
                   '"faber_wallet"}',
  'wallet_credentials': '{"key": '
                        '"faber_wallet_key"}'}
----------------------------------------------------------------------------------------------------


### (3) Acme(corporation) getting verinym

In [25]:
acme = {
    'name': 'Acme',
    'wallet_config': json.dumps({'id': 'acme_wallet'}),
    'wallet_credentials': json.dumps({'key': 'acme_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, acme)
print_result5 = IndyPrinter(acme)

In [26]:
print_result5("acme", use_pp = True)

----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'key': 'Ar8eC5VJAEVpAn6zjtwzoNFNgjzrzTJHqJk9t7SZW5vT',
  'name': 'Acme',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'wallet': 22,
  'wallet_config': '{"id": '
                   '"acme_wallet"}',
  'wallet_credentials': '{"key": '
                        '"acme_wallet_key"}'}
----------------------------------------------------------------------------------------------------


### (4) Thrift(bank) getting verinym

In [27]:
thrift = {
    'name': 'Thrift',
    'wallet_config': json.dumps({'id': 'thrift_wallet'}),
    'wallet_credentials': json.dumps({'key': 'thrift_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, thrift)
print_result6 = IndyPrinter(thrift)

In [28]:
print_result6("thrift", use_pp = True)

----------------------------------------------------------------------------------------------------
-----------------------------------------------thrift-----------------------------------------------
{ 'did': 'MQbsN1Q8Bvv5qNMWgVxXYS',
  'key': 'C86y4XhbiPZgvALYYbN3c61gJ8vWs3W67coGXVNByXfi',
  'name': 'Thrift',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'wallet': 26,
  'wallet_config': '{"id": '
                   '"thrift_wallet"}',
  'wallet_credentials': '{"key": '
                        '"thrift_wallet_key"}'}
----------------------------------------------------------------------------------------------------


<br><br><br><hr>

# 6. Credential Schema Setup

## - Schema 관련 함수 정의

In [29]:
async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)
    
async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    IndyPrinter(get_schema_request)("get_schema_request", use_pp = True) # DEBUG
    get_schema_response = await ensure_previous_request_applied(
        pool_handle, get_schema_request, lambda response: response['result']['data'] is not None)
    IndyPrinter(get_schema_response)("get_schema_response", use_pp = True) # DEBUG
    return_data = await ledger.parse_get_schema_response(get_schema_response)
    IndyPrinter(return_data)("return_data", use_pp = True) # DEBUG
    return return_data
#     return await ledger.parse_get_schema_response(get_schema_response)

### (1) Government Create "Transcript" Schema

In [30]:
transcript = {
    'name': 'Transcript',
    'version': '1.2',
    'attributes': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
}
(government['transcript_schema_id'], government['transcript_schema']) = \
    await anoncreds.issuer_create_schema(government['did'], transcript['name'], transcript['version'],
                                         json.dumps(transcript['attributes']))
transcript_schema_id = government['transcript_schema_id']

await send_schema(government['pool'], government['wallet'], government['did'], government['transcript_schema'])

print_result7 = IndyPrinter(government)

In [31]:
print_result8 = IndyPrinter(transcript)
print_result7("government", use_pp = True)
print_result8("transcript", use_pp = True)

----------------------------------------------------------------------------------------------------
---------------------------------------------government---------------------------------------------
{ 'did': 'DP92P2qRue3B7QaaM3ajWz',
  'key': '7kQmatTbAxGbEHn91Y8U7jsHLuznsvTt5PDgmxhhECpp',
  'name': 'Government',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'transcript_schema': '{"ver":"1.0","id":"DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2","name":"Transcript","version":"1.2","attrNames":["ssn","status","year","degree","average","first_name","last_name"],"seqNo":null}',
  'transcript_schema_id': 'DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2',
  'wallet': 15,
  'wallet_config': '{"id": '
                   '"government_wallet"}',
  'wallet_credentials': '{"key": '
                        '"government_wallet_key"}'}
----------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------

### (2) Government Create "Job-Certificate" Schema

In [32]:
job_certificate = {
    'name': 'Job-Certificate',
    'version': '0.2',
    'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
}
(government['job_certificate_schema_id'], government['job_certificate_schema']) = \
        await anoncreds.issuer_create_schema(government['did'], job_certificate['name'], job_certificate['version'], json.dumps(job_certificate['attributes']))

job_certificate_schema_id = government['job_certificate_schema_id']

await send_schema(government['pool'], government['wallet'], government['did'], government['job_certificate_schema'])

In [33]:
IndyPrinter(government)("government", use_pp = True)
IndyPrinter(transcript)("job_certificate_schema_id", use_pp = True)

----------------------------------------------------------------------------------------------------
---------------------------------------------government---------------------------------------------
{ 'did': 'DP92P2qRue3B7QaaM3ajWz',
  'job_certificate_schema': '{"ver":"1.0","id":"DP92P2qRue3B7QaaM3ajWz:2:Job-Certificate:0.2","name":"Job-Certificate","version":"0.2","attrNames":["employee_status","salary","first_name","experience","last_name"],"seqNo":null}',
  'job_certificate_schema_id': 'DP92P2qRue3B7QaaM3ajWz:2:Job-Certificate:0.2',
  'key': '7kQmatTbAxGbEHn91Y8U7jsHLuznsvTt5PDgmxhhECpp',
  'name': 'Government',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'transcript_schema': '{"ver":"1.0","id":"DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2","name":"Transcript","version":"1.2","attrNames":["ssn","status","year","degree","average","first_name","last_name"],"seqNo":null}',
  'transcript_schema_id': 'DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2',
  'wallet': 15,
  'wallet_config': '{"id": '
   

<br><br><br><hr>

# 7. Credential Definition Setup

## - Definition 관련 함수 정의

In [34]:
async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)

async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ensure_previous_request_applied(pool_handle, get_cred_def_request,
                                              lambda response: response['result']['data'] is not None)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)

### (1-1) Faber get "Transcript" schema from ledger

In [35]:
(faber['transcript_schema_id'], faber['transcript_schema']) = await get_schema(faber['pool'], faber['did'], transcript_schema_id)

----------------------------------------------------------------------------------------------------
-----------------------------------------get_schema_request-----------------------------------------
{ 'identifier': 'FyckJRmcaRJYeHukogSij5',
  'operation': { 'data': { 'name': 'Transcript',
                           'version': '1.2'},
                 'dest': 'DP92P2qRue3B7QaaM3ajWz',
                 'type': '107'},
  'protocolVersion': 2,
  'reqId': 1636378507167267928}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
------------------------------ensure_previous_request_applied-resposne------------------------------
{ 'op': 'REPLY',
  'result': { 'data': { 'attr_names': [ 'ssn',
                                        'average',
                                        'first_name',
                                        'degree',

In [36]:
IndyPrinter(faber)("faber", use_pp = True)

----------------------------------------------------------------------------------------------------
-----------------------------------------------faber------------------------------------------------
{ 'did': 'FyckJRmcaRJYeHukogSij5',
  'key': '9ARg7ByY8TUaYufZ1ezZwe7ArET1X3vsXm43NDbyda3y',
  'name': 'Faber',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'transcript_schema': '{"ver":"1.0","id":"DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2","name":"Transcript","version":"1.2","attrNames":["degree","last_name","average","year","status","first_name","ssn"],"seqNo":11}',
  'transcript_schema_id': 'DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2',
  'wallet': 19,
  'wallet_config': '{"id": '
                   '"faber_wallet"}',
  'wallet_credentials': '{"key": '
                        '"faber_wallet_key"}'}
----------------------------------------------------------------------------------------------------


### (1-2) Faber create and store in wallet "Faber Transcript" credential definition

In [37]:
transcript_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": False}
}
(faber['transcript_cred_def_id'], faber['transcript_cred_def']) = await anoncreds.issuer_create_and_store_credential_def(faber['wallet'], faber['did'],
                                                                                                                         faber['transcript_schema'], transcript_cred_def['tag'],
                                                                                                                         transcript_cred_def['type'],
                                                                                                                         json.dumps(transcript_cred_def['config']))

In [38]:
IndyPrinter(transcript_cred_def)("transcript_cred_def", use_pp = True)
IndyPrinter(faber)("faber", use_pp = True)

----------------------------------------------------------------------------------------------------
----------------------------------------transcript_cred_def-----------------------------------------
{ 'config': { 'support_revocation': False},
  'tag': 'TAG1',
  'type': 'CL'}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
-----------------------------------------------faber------------------------------------------------
{ 'did': 'FyckJRmcaRJYeHukogSij5',
  'key': '9ARg7ByY8TUaYufZ1ezZwe7ArET1X3vsXm43NDbyda3y',
  'name': 'Faber',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'transcript_cred_def': '{"ver":"1.0","id":"FyckJRmcaRJYeHukogSij5:3:CL:11:TAG1","schemaId":"11","type":"CL","tag":"TAG1","value":{"primary":{"n":"10199611637626751001637838379652162260610132347605190338086359114788639345923401730842493975612683370789811135007156031

### (1-3) Faber send "Faber Transcript" credential definition to ledger

In [39]:
await send_cred_def(faber['pool'], faber['wallet'], faber['did'], faber['transcript_cred_def'])

### (2-1) Acme get "Job-Certificate" schema from ledger

In [40]:
(acme['job_certificate_schema_id'], acme['job_certificate_schema']) = await get_schema(acme['pool'], acme['did'], job_certificate_schema_id)

----------------------------------------------------------------------------------------------------
-----------------------------------------get_schema_request-----------------------------------------
{ 'identifier': 'K4tCkhb6nUJyYjmiaFDeo7',
  'operation': { 'data': { 'name': 'Job-Certificate',
                           'version': '0.2'},
                 'dest': 'DP92P2qRue3B7QaaM3ajWz',
                 'type': '107'},
  'protocolVersion': 2,
  'reqId': 1636378511024173604}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
------------------------------ensure_previous_request_applied-resposne------------------------------
{ 'op': 'REPLY',
  'result': { 'data': { 'attr_names': [ 'salary',
                                        'experience',
                                        'employee_status',
                                 

In [41]:
IndyPrinter(acme)("acme", use_pp = True)

----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'job_certificate_schema': '{"ver":"1.0","id":"DP92P2qRue3B7QaaM3ajWz:2:Job-Certificate:0.2","name":"Job-Certificate","version":"0.2","attrNames":["first_name","salary","employee_status","last_name","experience"],"seqNo":12}',
  'job_certificate_schema_id': 'DP92P2qRue3B7QaaM3ajWz:2:Job-Certificate:0.2',
  'key': 'Ar8eC5VJAEVpAn6zjtwzoNFNgjzrzTJHqJk9t7SZW5vT',
  'name': 'Acme',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'wallet': 22,
  'wallet_config': '{"id": '
                   '"acme_wallet"}',
  'wallet_credentials': '{"key": '
                        '"acme_wallet_key"}'}
----------------------------------------------------------------------------------------------------


### (2-2) Acme create and store in wallet "Acme Job-Certificate" credential definition

In [42]:
job_certificate_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": True}
}
(acme['job_certificate_cred_def_id'], acme['job_certificate_cred_def']) = await anoncreds.issuer_create_and_store_credential_def(acme['wallet'], acme['did'],
                                                                                                                                 acme['job_certificate_schema'],
                                                                                                                                 job_certificate_cred_def['tag'],
                                                                                                                                 job_certificate_cred_def['type'],
                                                                                                                                 json.dumps(job_certificate_cred_def['config']))

In [43]:
IndyPrinter(transcript_cred_def)("job_certificate_cred_def", use_pp = True)
IndyPrinter(acme)("acme", use_pp = True)

----------------------------------------------------------------------------------------------------
--------------------------------------job_certificate_cred_def--------------------------------------
{ 'config': { 'support_revocation': False},
  'tag': 'TAG1',
  'type': 'CL'}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'job_certificate_cred_def': '{"ver":"1.0","id":"K4tCkhb6nUJyYjmiaFDeo7:3:CL:12:TAG1","schemaId":"12","type":"CL","tag":"TAG1","value":{"primary":{"n":"10237026792136609982118954642645231360202530127008855033274716134249233204814903009319644732191168459193353862540725962687397571155463398783507722518727239636404695073931202301137564373095096469542252624334114372907324995075745540

### (2-3) Acme send "Acme Job-Certificate" credential definition to ledger

In [44]:
await send_cred_def(acme['pool'], acme['wallet'], acme['did'], acme['job_certificate_cred_def'])

### (2-4) Acme create revocation registry

In [45]:
acme['tails_writer_config'] = json.dumps({'base_dir': "/tmp/indy_acme_tails", 'uri_pattern': ''})
tails_writer = await blob_storage.open_writer('default', acme['tails_writer_config'])
(acme['revoc_reg_id'], acme['revoc_reg_def'], acme['revoc_reg_entry']) = await anoncreds.issuer_create_and_store_revoc_reg(acme['wallet'], acme['did'], 'CL_ACCUM', 'TAG1',
                                                                                                                           acme['job_certificate_cred_def_id'],
                                                                                                                           json.dumps({'max_cred_num': 5, 'issuance_type': 'ISSUANCE_ON_DEMAND'}),
                                                                                                                           tails_writer)

In [46]:
IndyPrinter(acme)("acme")

----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'job_certificate_cred_def': '{"ver":"1.0","id":"K4tCkhb6nUJyYjmiaFDeo7:3:CL:12:TAG1","schemaId":"12","type":"CL","tag":"TAG1","value":{"primary":{"n":"1023702679213660998211895464264523136020253012700885503327471613424923320481490300931964473219116845919335386254072596268739757115546339878350772251872723963640469507393120230113756437309509646954225262433411437290732499507574554074821716797758311440549199994548436432304315531856142474173680297879461933932511459491091247053204385810437216795128662206211421247048429875804767018352170225345023507618364385259771325108599545207095576462828245675473499795624841135634156983080727758320482962342434303656713250224501753779499339061689381941638144183049264671238568499736713419511533694318771606966294989799216117

### (2-5) Acme post revocation registry definition to ledger

In [47]:
acme['revoc_reg_def_request'] = await ledger.build_revoc_reg_def_request(acme['did'], acme['revoc_reg_def'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_def_request'])
print()

In [48]:
IndyPrinter(acme)("acme")

----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'job_certificate_cred_def': '{"ver":"1.0","id":"K4tCkhb6nUJyYjmiaFDeo7:3:CL:12:TAG1","schemaId":"12","type":"CL","tag":"TAG1","value":{"primary":{"n":"1023702679213660998211895464264523136020253012700885503327471613424923320481490300931964473219116845919335386254072596268739757115546339878350772251872723963640469507393120230113756437309509646954225262433411437290732499507574554074821716797758311440549199994548436432304315531856142474173680297879461933932511459491091247053204385810437216795128662206211421247048429875804767018352170225345023507618364385259771325108599545207095576462828245675473499795624841135634156983080727758320482962342434303656713250224501753779499339061689381941638144183049264671238568499736713419511533694318771606966294989799216117

### (2-6) Acme post revocation registry entry to ledger

In [49]:
acme['revoc_reg_entry_request'] = await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM', acme['revoc_reg_entry'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_request'])
print()

In [50]:
IndyPrinter(acme)("acme")

----------------------------------------------------------------------------------------------------
------------------------------------------------acme------------------------------------------------
{ 'did': 'K4tCkhb6nUJyYjmiaFDeo7',
  'job_certificate_cred_def': '{"ver":"1.0","id":"K4tCkhb6nUJyYjmiaFDeo7:3:CL:12:TAG1","schemaId":"12","type":"CL","tag":"TAG1","value":{"primary":{"n":"1023702679213660998211895464264523136020253012700885503327471613424923320481490300931964473219116845919335386254072596268739757115546339878350772251872723963640469507393120230113756437309509646954225262433411437290732499507574554074821716797758311440549199994548436432304315531856142474173680297879461933932511459491091247053204385810437216795128662206211421247048429875804767018352170225345023507618364385259771325108599545207095576462828245675473499795624841135634156983080727758320482962342434303656713250224501753779499339061689381941638144183049264671238568499736713419511533694318771606966294989799216117

<br><br><br><hr>

# 8. Alice Create DID and wallet

In [51]:
alice = {
    'name': 'Alice',
    'wallet_config': json.dumps({'id': 'alice_wallet'}),
    'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
    'pool': pool_['handle'],
}
await create_wallet(alice)
(alice['did'], alice['key']) = await did.create_and_store_my_did(alice['wallet'], "{}")

In [52]:
IndyPrinter(alice)("alice")

----------------------------------------------------------------------------------------------------
-----------------------------------------------alice------------------------------------------------
{ 'did': '3gq8nHycjTVwX966dDZ1Ka',
  'key': '2Tw6EpUja7M9HzyaNWvZ9X9RgHmd8SeE5PCkDriYCS8T',
  'name': 'Alice',
  'pool': 10,
  'wallet': 44,
  'wallet_config': '{"id": '
                   '"alice_wallet"}',
  'wallet_credentials': '{"key": '
                        '"alice_wallet_key"}'}
----------------------------------------------------------------------------------------------------


<br><br><br><hr>

# 9. Alice Get "Transcript" Credential

### (1) Faber create "Transcript" credential offer for alice

In [53]:
faber['transcript_cred_offer'] = await anoncreds.issuer_create_credential_offer(faber['wallet'], faber['transcript_cred_def_id'])
alice['transcript_cred_offer'] = faber['transcript_cred_offer']
transcript_cred_offer_object = json.loads(alice['transcript_cred_offer'])
alice['transcript_schema_id'] = transcript_cred_offer_object['schema_id']
alice['transcript_cred_def_id'] = transcript_cred_offer_object['cred_def_id']

In [54]:
IndyPrinter(faber)("faber")
IndyPrinter(alice)("alice")

----------------------------------------------------------------------------------------------------
-----------------------------------------------faber------------------------------------------------
{ 'did': 'FyckJRmcaRJYeHukogSij5',
  'key': '9ARg7ByY8TUaYufZ1ezZwe7ArET1X3vsXm43NDbyda3y',
  'name': 'Faber',
  'pool': 10,
  'role': 'TRUST_ANCHOR',
  'transcript_cred_def': '{"ver":"1.0","id":"FyckJRmcaRJYeHukogSij5:3:CL:11:TAG1","schemaId":"11","type":"CL","tag":"TAG1","value":{"primary":{"n":"1019961163762675100163783837965216226061013234760519033808635911478863934592340173084249397561268337078981113500715603133890169480599486689177089921097092837099605015653820238391825175711996510385456755738571431157138191211955445594627274680674421440248308348281434951394730355913217004432943995872743757594493363066844922316879356099758196430603669836274425185485278285020508584532648146666729845561709719764841528521577432611340334225614877783739390390893233064984056464478527043728988096778020620

### (2) Alice create and store master secret in wallet

In [55]:
alice['master_secret_id'] = await anoncreds.prover_create_master_secret(alice['wallet'], None)

In [56]:
IndyPrinter(alice)('alice')

----------------------------------------------------------------------------------------------------
-----------------------------------------------alice------------------------------------------------
{ 'did': '3gq8nHycjTVwX966dDZ1Ka',
  'key': '2Tw6EpUja7M9HzyaNWvZ9X9RgHmd8SeE5PCkDriYCS8T',
  'master_secret_id': 'ba323595-bf64-496a-8013-2531cd918b7b',
  'name': 'Alice',
  'pool': 10,
  'transcript_cred_def_id': 'FyckJRmcaRJYeHukogSij5:3:CL:11:TAG1',
  'transcript_cred_offer': '{"schema_id":"DP92P2qRue3B7QaaM3ajWz:2:Transcript:1.2","cred_def_id":"FyckJRmcaRJYeHukogSij5:3:CL:11:TAG1","key_correctness_proof":{"c":"90145142242100077791032519748739980059037656814133943346236467570969043011235","xz_cap":"57155632940030311554822781649231464742151414152513946283332917052242056392741245418805871576647828517259579049708466356738629467374755237566723118087274180418443656152145622013223228812209770581517623404322294127703688103349541278592075501275481293987159333930099180888799345933916666264528

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| START ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [ ]:
# alice = {
#     'name': 'Alice',
#     'wallet_config': json.dumps({'id': 'alice_wallet'}),
#     'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
#     'pool': pool_['handle'],
# }

await did.set_endpoint_for_did(acme['wallet'], acme['did'], 'http://localhost:1234', acme['key'])
await did.get_endpoint_for_did(acme['wallet'], acme['pool'], acme['did'])

In [ ]:
await did.set_endpoint_for_did(acme['wallet'], acme['did'], 'http://localhost:1235', acme['key'])
await did.get_endpoint_for_did(acme['wallet'], acme['pool'], acme['did'])

In [ ]:
acme

In [57]:
alice['did']

'3gq8nHycjTVwX966dDZ1Ka'

In [75]:
# req_ddo = await ledger.build_get_ddo_request(steward['did'], acme['did'])
req_ddo = await ledger.build_get_ddo_request(steward['did'], 'FyckJRmcaRJYeHukogSij5')
IndyPrinter(req_ddo)(use_pp=True)

----------------------------------------------------------------------------------------------------
{ 'identifier': 'Th7MpTaRZVRYnPiabds81Y',
  'operation': { 'dest': 'FyckJRmcaRJYeHukogSij5',
                 'type': '120'},
  'protocolVersion': 2,
  'reqId': 1636379185676912978}
----------------------------------------------------------------------------------------------------


In [76]:
await ledger.sign_and_submit_request(faber['pool'], faber['wallet'], 'FyckJRmcaRJYeHukogSij5', req_ddo)

'{"op":"REQNACK","identifier":"Th7MpTaRZVRYnPiabds81Y","reqId":1636379185676912978,"reason":"client request invalid: InvalidClientRequest(\'Th7MpTaRZVRYnPiabds81Y\', 1636379185676912978, \'validation error [ClientAuthRuleOperation]: missed fields - constraint, auth_type, field, new_value, auth_action. \')"}'

In [73]:
await ledger.submit_request(alice['pool'], req_ddo)

'{"op":"REQNACK","identifier":"Th7MpTaRZVRYnPiabds81Y","reqId":1636378900354477127,"reason":"client request invalid: InvalidClientRequest(\'Th7MpTaRZVRYnPiabds81Y\', 1636378900354477127, \'validation error [ClientAuthRuleOperation]: missed fields - auth_type, constraint, auth_action, field, new_value. \')"}'

In [74]:
await did.list_my_dids_with_meta(alice['wallet'])

'[{"did":"4FWeoW8EkSEkwpoU82DWSE","verkey":"2mkDHcVrjmmpqg36dBNkVx9LC8UYxjAHzixBahZgmtWg","tempVerkey":null,"metadata":"{}"},{"did":"R1MbDGVf9cotKSiasb9EVR","verkey":"E5t61SzZABCdUx632YEkxTUUdQQfLasetVmg4gnerVTV","tempVerkey":null,"metadata":"{\\"endpoint\\": \\"https://myhost:8021\\"}"},{"did":"3gq8nHycjTVwX966dDZ1Ka","verkey":"2Tw6EpUja7M9HzyaNWvZ9X9RgHmd8SeE5PCkDriYCS8T","tempVerkey":null,"metadata":null},{"did":"7sLxuDCJ5aqUNcGvGpUEEr","verkey":"4k7SqryGrD3PafcY4K4aXJnrr4m44ks6BxGFeKjDMdpp","tempVerkey":null,"metadata":null}]'

In [ ]:
await did.key_for_did(acme['pool'], acme['wallet'], acme['did'])

In [ ]:
await did.create_key(acme['wallet'],'{}')

In [ ]:
await did.list_my_dids_with_meta(acme['wallet'])

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| END ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

### (3) Alice get "Faber Transcript" credential definition from ledger

In [ ]:
(alice['faber_transcript_cred_def_id'], alice['faber_transcript_cred_def']) = await get_cred_def(alice['pool'], alice['did'], alice['transcript_cred_def_id'])

### (4) Alice create "Transcript" credential request for Faber

In [ ]:
(alice['transcript_cred_request'], alice['transcript_cred_request_metadata']) = \
    await anoncreds.prover_create_credential_req(alice['wallet'], alice['did'],
                                                    alice['transcript_cred_offer'], alice['faber_transcript_cred_def'],
                                                    alice['master_secret_id'])

### (5) Alice send "Transcript" credential request for Faber

In [ ]:
faber['transcript_cred_request'] = alice['transcript_cred_request']

### (6) Faber create "Transcript" credential for Alice

In [ ]:
faber['alice_transcript_cred_values'] = json.dumps({
    "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
    "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
    "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
    "status": {"raw": "graduated", "encoded": "2213454313412354"},
    "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
    "year": {"raw": "2015", "encoded": "2015"},
    "average": {"raw": "5", "encoded": "5"}
})

faber['transcript_cred'], _, _ = \
    await anoncreds.issuer_create_credential(faber['wallet'], faber['transcript_cred_offer'],
                                                faber['transcript_cred_request'],
                                                faber['alice_transcript_cred_values'], None, None)

### (7) Faber send "Transcript" credential to Alice

In [ ]:
alice['transcript_cred'] = faber['transcript_cred']

### (8) Alice store "Transcript" credential from Faber

In [ ]:
_, alice['transcript_cred_def'] = await get_cred_def(alice['pool'], alice['did'],
                                                        alice['transcript_cred_def_id'])

await anoncreds.prover_store_credential(alice['wallet'], None, alice['transcript_cred_request_metadata'],
                                        alice['transcript_cred'], alice['transcript_cred_def'], None)
print()

<br><br><br><hr>

# 10. Apply For the Job With Acme - Transcript Proving

## - Credential 검증 관련 함수 정의

In [ ]:
async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']


def get_timestamp_for_attribute(cred_for_attribute, revoc_states):
    if cred_for_attribute['rev_reg_id'] in revoc_states:
        return int(next(iter(revoc_states[cred_for_attribute['rev_reg_id']])))
    else:
        return None


async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor, timestamp_from=None,
                                          timestamp_to=None):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        # logger.info("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        # logger.info("\"{}\" -> Get Claim Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_id' in item and item['rev_reg_id'] is not None:
            # Create Revocations States
            # logger.info("\"{}\" -> Get Revocation Registry Definition from Ledger".format(actor))
            get_revoc_reg_def_request = await ledger.build_get_revoc_reg_def_request(_did, item['rev_reg_id'])

            get_revoc_reg_def_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_def_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_def_json) = await ledger.parse_get_revoc_reg_def_response(get_revoc_reg_def_response)

            # logger.info("\"{}\" -> Get Revocation Registry Delta from Ledger".format(actor))
            if not timestamp_to: timestamp_to = int(time.time())
            get_revoc_reg_delta_request = \
                await ledger.build_get_revoc_reg_delta_request(_did, item['rev_reg_id'], timestamp_from, timestamp_to)
            get_revoc_reg_delta_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_delta_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_delta_json, t) = \
                await ledger.parse_get_revoc_reg_delta_response(get_revoc_reg_delta_response)

            tails_reader_config = json.dumps(
                {'base_dir': dirname(json.loads(revoc_reg_def_json)['value']['tailsLocation']),
                 'uri_pattern': ''})
            blob_storage_reader_cfg_handle = await blob_storage.open_reader('default', tails_reader_config)

            # logger.info('%s - Create Revocation State', actor)
            rev_state_json = \
                await anoncreds.create_revocation_state(blob_storage_reader_cfg_handle, revoc_reg_def_json,
                                                        revoc_reg_delta_json, t, item['cred_rev_id'])
            rev_states[rev_reg_id] = {t: json.loads(rev_state_json)}

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)


async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor, timestamp=None):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        # logger.info("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        # logger.info("\"{}\" -> Get Claim Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_id' in item and item['rev_reg_id'] is not None:
            # Get Revocation Definitions and Revocation Registries
            # logger.info("\"{}\" -> Get Revocation Definition from Ledger".format(actor))
            get_revoc_reg_def_request = await ledger.build_get_revoc_reg_def_request(_did, item['rev_reg_id'])

            get_revoc_reg_def_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_def_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_def_json) = await ledger.parse_get_revoc_reg_def_response(get_revoc_reg_def_response)

            # logger.info("\"{}\" -> Get Revocation Registry from Ledger".format(actor))
            if not timestamp: timestamp = item['timestamp']
            get_revoc_reg_request = \
                await ledger.build_get_revoc_reg_request(_did, item['rev_reg_id'], timestamp)
            get_revoc_reg_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, rev_reg_json, timestamp2) = await ledger.parse_get_revoc_reg_response(get_revoc_reg_response)

            rev_regs[rev_reg_id] = {timestamp2: json.loads(rev_reg_json)}
            rev_reg_defs[rev_reg_id] = json.loads(revoc_reg_def_json)

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)

### (1) Acme create "Job-Application" proof request

In [ ]:
nonce = await anoncreds.generate_nonce()

acme['job_application_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Job-Application',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {
            'name': 'first_name'
        },
        'attr2_referent': {
            'name': 'last_name'
        },
        'attr3_referent': {
            'name': 'degree',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr4_referent': {
            'name': 'status',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr5_referent': {
            'name': 'ssn',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr6_referent': {
            'name': 'phone_number'
        }
    },
    'requested_predicates': {
        'predicate1_referent': {
            'name': 'average',
            'p_type': '>=',
            'p_value': 4,
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        }
    }
})

### (2) Acme send "Job-Application" proof request to Alice

In [ ]:
alice['job_application_proof_request'] = acme['job_application_proof_request']

### (3) Alice get credentials for "Job-Application" proof request

In [ ]:
search_for_job_application_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['job_application_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr1_referent')
cred_for_attr2 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr2_referent')
cred_for_attr3 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr3_referent')
cred_for_attr4 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr4_referent')
cred_for_attr5 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr5_referent')
cred_for_predicate1 = \
    await get_credential_for_referent(search_for_job_application_proof_request, 'predicate1_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_job_application_proof_request)

alice['creds_for_job_application_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                            cred_for_attr2['referent']: cred_for_attr2,
                                            cred_for_attr3['referent']: cred_for_attr3,
                                            cred_for_attr4['referent']: cred_for_attr4,
                                            cred_for_attr5['referent']: cred_for_attr5,
                                            cred_for_predicate1['referent']: cred_for_predicate1}

alice['schemas_for_job_application'], alice['cred_defs_for_job_application'], \
alice['revoc_states_for_job_application'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                            alice['creds_for_job_application_proof'], alice['name'])

### (4) Alice create "Job-certificate" proof

In [ ]:
alice['job_application_requested_creds'] = json.dumps({
    'self_attested_attributes': {
        'attr1_referent': 'Alice',
        'attr2_referent': 'Garcia',
        'attr6_referent': '123-45-6789'
    },
    'requested_attributes': {
        'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True},
        'attr4_referent': {'cred_id': cred_for_attr4['referent'], 'revealed': True},
        'attr5_referent': {'cred_id': cred_for_attr5['referent'], 'revealed': True},
    },
    'requested_predicates': {'predicate1_referent': {'cred_id': cred_for_predicate1['referent']}}
})

alice['job_application_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['job_application_proof_request'],
                                        alice['job_application_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_job_application'],
                                        alice['cred_defs_for_job_application'],
                                        alice['revoc_states_for_job_application'])

### (5) Alice send "Job-Application" proof to Acme

In [ ]:
acme['job_application_proof'] = alice['job_application_proof']

job_application_proof_object = json.loads(acme['job_application_proof'])

acme['schemas_for_job_application'], acme['cred_defs_for_job_application'], \
acme['revoc_ref_defs_for_job_application'], acme['revoc_regs_for_job_application'] = \
    await verifier_get_entities_from_ledger(acme['pool'], acme['did'],
                                            job_application_proof_object['identifiers'], acme['name'])

### (6) Acme verify "Job-Application" proof from Alice

In [ ]:
assert 'Bachelor of Science, Marketing' == \
        job_application_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']
assert 'graduated' == \
        job_application_proof_object['requested_proof']['revealed_attrs']['attr4_referent']['raw']
assert '123-45-6789' == \
        job_application_proof_object['requested_proof']['revealed_attrs']['attr5_referent']['raw']

assert 'Alice' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr1_referent']
assert 'Garcia' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr2_referent']
assert '123-45-6789' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr6_referent']

assert await anoncreds.verifier_verify_proof(acme['job_application_proof_request'], acme['job_application_proof'],
                                                acme['schemas_for_job_application'],
                                                acme['cred_defs_for_job_application'],
                                                acme['revoc_ref_defs_for_job_application'],
                                                acme['revoc_regs_for_job_application'])

<br><br><br><hr>

# 11. Apply for the job with Acme - Getting Job-Certificate Credential

In [ ]:
# logger.info("\"Acme\" -> Create \"Job-Certificate\" Credential Offer for Alice")

In [ ]:
acme['job_certificate_cred_offer'] = \
    await anoncreds.issuer_create_credential_offer(acme['wallet'], acme['job_certificate_cred_def_id'])

In [ ]:
# logger.info("\"Acme\" -> Send \"Job-Certificate\" Credential Offer to Alice")

In [ ]:
alice['job_certificate_cred_offer'] = acme['job_certificate_cred_offer']
job_certificate_cred_offer_object = json.loads(alice['job_certificate_cred_offer'])

In [ ]:
# logger.info("\"Alice\" -> Get \"Acme Job-Certificate\" Credential Definition from Ledger")

In [ ]:
(alice['acme_job_certificate_cred_def_id'], alice['acme_job_certificate_cred_def']) = \
    await get_cred_def(alice['pool'], alice['did'], job_certificate_cred_offer_object['cred_def_id'])

In [ ]:
# logger.info("\"Alice\" -> Create and store in Wallet \"Job-Certificate\" Credential Request for Acme")

In [ ]:
(alice['job_certificate_cred_request'], alice['job_certificate_cred_request_metadata']) = \
    await anoncreds.prover_create_credential_req(alice['wallet'], alice['did'],
                                                    alice['job_certificate_cred_offer'],
                                                    alice['acme_job_certificate_cred_def'], alice['master_secret_id'])

In [ ]:
# logger.info("\"Alice\" -> Send \"Job-Certificate\" Credential Request to Acme")

In [ ]:
alice['job_certificate_cred_values'] = json.dumps({
    "first_name": {"raw": "Alice", "encoded": "245712572474217942457235975012103335"},
    "last_name": {"raw": "Garcia", "encoded": "312643218496194691632153761283356127"},
    "employee_status": {"raw": "Permanent", "encoded": "2143135425425143112321314321"},
    "salary": {"raw": "2400", "encoded": "2400"},
    "experience": {"raw": "10", "encoded": "10"}
})
acme['job_certificate_cred_request'] = alice['job_certificate_cred_request']
acme['job_certificate_cred_values'] = alice['job_certificate_cred_values']

In [ ]:
# logger.info("\"Acme\" -> Create \"Job-Certificate\" Credential for Alice")

In [ ]:
acme['blob_storage_reader_cfg_handle'] = await blob_storage.open_reader('default', acme['tails_writer_config'])
acme['job_certificate_cred'], acme['job_certificate_cred_rev_id'], acme['alice_cert_rev_reg_delta'] = \
    await anoncreds.issuer_create_credential(acme['wallet'], acme['job_certificate_cred_offer'],
                                                acme['job_certificate_cred_request'],
                                                acme['job_certificate_cred_values'],
                                                acme['revoc_reg_id'],
                                                acme['blob_storage_reader_cfg_handle'])

In [ ]:
# logger.info("\"Acme\" -> Post Revocation Registry Delta to Ledger")

In [ ]:
acme['revoc_reg_entry_req'] = \
    await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM',
                                                acme['alice_cert_rev_reg_delta'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_req'])
print()

In [ ]:
# logger.info("\"Acme\" -> Send \"Job-Certificate\" Credential to Alice")

In [ ]:
alice['job_certificate_cred'] = acme['job_certificate_cred']
job_certificate_cred_object = json.loads(alice['job_certificate_cred'])

In [ ]:
# logger.info("\"Alice\" -> Gets RevocationRegistryDefinition for \"Job-Certificate\" Credential from Acme")

In [ ]:
alice['acme_revoc_reg_des_req'] = \
    await ledger.build_get_revoc_reg_def_request(alice['did'],
                                                    job_certificate_cred_object['rev_reg_id'])
alice['acme_revoc_reg_des_resp'] = \
    await ensure_previous_request_applied(alice['pool'], alice['acme_revoc_reg_des_req'],
                                            lambda response: response['result']['data'] is not None)
(alice['acme_revoc_reg_def_id'], alice['acme_revoc_reg_def_json']) = \
    await ledger.parse_get_revoc_reg_def_response(alice['acme_revoc_reg_des_resp'])

In [ ]:
# logger.info("\"Alice\" -> Store \"Job-Certificate\" Credential")

In [ ]:
await anoncreds.prover_store_credential(alice['wallet'], None, alice['job_certificate_cred_request_metadata'],
                                        alice['job_certificate_cred'],
                                        alice['acme_job_certificate_cred_def'], alice['acme_revoc_reg_def_json'])
print()

<br><br><br><hr>

# 12. Apply for the loan with Thrift - Job-Certificate proving

In [ ]:
# logger.info("\"Thrift\" -> Create \"Loan-Application-Basic\" Proof Request")
nonce = await anoncreds.generate_nonce()
thrift['apply_loan_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Loan-Application-Basic',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {
            'name': 'employee_status',
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        }
    },
    'requested_predicates': {
        'predicate1_referent': {
            'name': 'salary',
            'p_type': '>=',
            'p_value': 2000,
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        },
        'predicate2_referent': {
            'name': 'experience',
            'p_type': '>=',
            'p_value': 1,
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        }
    },
    'non_revoked': {'to': int(time.time())}
})

In [ ]:
# logger.info("\"Thrift\" -> Send \"Loan-Application-Basic\" Proof Request to Alice")
alice['apply_loan_proof_request'] = thrift['apply_loan_proof_request']

In [ ]:
# logger.info("\"Alice\" -> Get credentials for \"Loan-Application-Basic\" Proof Request")
search_for_apply_loan_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['apply_loan_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'attr1_referent')
cred_for_predicate1 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                        'predicate1_referent')
cred_for_predicate2 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                        'predicate2_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_proof_request)

alice['creds_for_apply_loan_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                        cred_for_predicate1['referent']: cred_for_predicate1,
                                        cred_for_predicate2['referent']: cred_for_predicate2}

requested_timestamp = int(json.loads(thrift['apply_loan_proof_request'])['non_revoked']['to'])
alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'], alice['revoc_states_for_loan_app'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                            alice['creds_for_apply_loan_proof'],
                                            alice['name'], None, requested_timestamp)

In [ ]:
# logger.info("\"Alice\" -> Create \"Loan-Application-Basic\" Proof")
revoc_states_for_loan_app = json.loads(alice['revoc_states_for_loan_app'])
timestamp_for_attr1 = get_timestamp_for_attribute(cred_for_attr1, revoc_states_for_loan_app)
timestamp_for_predicate1 = get_timestamp_for_attribute(cred_for_predicate1, revoc_states_for_loan_app)
timestamp_for_predicate2 = get_timestamp_for_attribute(cred_for_predicate2, revoc_states_for_loan_app)
alice['apply_loan_requested_creds'] = json.dumps({
    'self_attested_attributes': {},
    'requested_attributes': {
        'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True,
                            'timestamp': timestamp_for_attr1}
    },
    'requested_predicates': {
        'predicate1_referent': {'cred_id': cred_for_predicate1['referent'],
                                'timestamp': timestamp_for_predicate1},
        'predicate2_referent': {'cred_id': cred_for_predicate2['referent'],
                                'timestamp': timestamp_for_predicate2}
    }
})
alice['apply_loan_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['apply_loan_proof_request'],
                                        alice['apply_loan_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'],
                                        alice['revoc_states_for_loan_app'])

In [ ]:
# logger.info("\"Alice\" -> Send \"Loan-Application-Basic\" Proof to Thrift")
thrift['apply_loan_proof'] = alice['apply_loan_proof']
apply_loan_proof_object = json.loads(thrift['apply_loan_proof'])

In [ ]:
# logger.info("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying")

thrift['schemas_for_loan_app'], thrift['cred_defs_for_loan_app'], thrift['revoc_defs_for_loan_app'], \
thrift['revoc_regs_for_loan_app'] = \
    await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                            apply_loan_proof_object['identifiers'],
                                            thrift['name'], requested_timestamp)

In [ ]:
# logger.info("\"Thrift\" -> Verify \"Loan-Application-Basic\" Proof from Alice")
assert 'Permanent' == apply_loan_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']

assert await anoncreds.verifier_verify_proof(thrift['apply_loan_proof_request'],
                                                thrift['apply_loan_proof'],
                                                thrift['schemas_for_loan_app'],
                                                thrift['cred_defs_for_loan_app'],
                                                thrift['revoc_defs_for_loan_app'],
                                                thrift['revoc_regs_for_loan_app'])

<br><br><br><hr>

# 13. Apply for the loan with Thrift - Transcript and Job-Certificate proving

In [ ]:
# logger.info("\"Thrift\" -> Create \"Loan-Application-KYC\" Proof Request")
nonce = await anoncreds.generate_nonce()
thrift['apply_loan_kyc_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Loan-Application-KYC',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {'name': 'first_name'},
        'attr2_referent': {'name': 'last_name'},
        'attr3_referent': {'name': 'ssn'}
    },
    'requested_predicates': {}
})

In [ ]:
# logger.info("\"Thrift\" -> Send \"Loan-Application-KYC\" Proof Request to Alice")
alice['apply_loan_kyc_proof_request'] = thrift['apply_loan_kyc_proof_request']

In [ ]:
# logger.info("\"Alice\" -> Get credentials for \"Loan-Application-KYC\" Proof Request")

search_for_apply_loan_kyc_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['apply_loan_kyc_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr1_referent')
cred_for_attr2 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr2_referent')
cred_for_attr3 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr3_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_kyc_proof_request)

alice['creds_for_apply_loan_kyc_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                            cred_for_attr2['referent']: cred_for_attr2,
                                            cred_for_attr3['referent']: cred_for_attr3}

alice['schemas_for_loan_kyc_app'], alice['cred_defs_for_loan_kyc_app'], alice['revoc_states_for_loan_kyc_app'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                            alice['creds_for_apply_loan_kyc_proof'], alice['name'], )

In [ ]:
# logger.info("\"Alice\" -> Create \"Loan-Application-KYC\" Proof")
revoc_states_for_loan_app = json.loads(alice['revoc_states_for_loan_kyc_app'])
timestamp_for_attr1 = get_timestamp_for_attribute(cred_for_attr1, revoc_states_for_loan_app)
timestamp_for_attr2 = get_timestamp_for_attribute(cred_for_attr2, revoc_states_for_loan_app)
timestamp_for_attr3 = get_timestamp_for_attribute(cred_for_attr3, revoc_states_for_loan_app)
alice['apply_loan_kyc_requested_creds'] = json.dumps({
    'self_attested_attributes': {},
    'requested_attributes': {
        'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True,
                            'timestamp': timestamp_for_attr1},
        'attr2_referent': {'cred_id': cred_for_attr2['referent'], 'revealed': True,
                            'timestamp': timestamp_for_attr2},
        'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True,
                            'timestamp': timestamp_for_attr3}
    },
    'requested_predicates': {}
})

alice['apply_loan_kyc_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['apply_loan_kyc_proof_request'],
                                        alice['apply_loan_kyc_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_loan_kyc_app'], alice['cred_defs_for_loan_kyc_app'],
                                        alice['revoc_states_for_loan_kyc_app'])

In [ ]:
# logger.info("\"Alice\" -> Send \"Loan-Application-KYC\" Proof to Thrift")
thrift['apply_loan_kyc_proof'] = alice['apply_loan_kyc_proof']
apply_loan_kyc_proof_object = json.loads(thrift['apply_loan_kyc_proof'])

In [ ]:
# logger.info("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying")

thrift['schemas_for_loan_kyc_app'], thrift['cred_defs_for_loan_kyc_app'], thrift['revoc_defs_for_loan_kyc_app'], \
thrift['revoc_regs_for_loan_kyc_app'] = \
    await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                            apply_loan_kyc_proof_object['identifiers'], thrift['name'])

In [ ]:
# logger.info("\"Thrift\" -> Verify \"Loan-Application-KYC\" Proof from Alice")
assert 'Alice' == \
        apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']
assert 'Garcia' == \
        apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr2_referent']['raw']
assert '123-45-6789' == \
        apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']

assert await anoncreds.verifier_verify_proof(thrift['apply_loan_kyc_proof_request'],
                                                thrift['apply_loan_kyc_proof'],
                                                thrift['schemas_for_loan_kyc_app'],
                                                thrift['cred_defs_for_loan_kyc_app'],
                                                thrift['revoc_defs_for_loan_kyc_app'],
                                                thrift['revoc_regs_for_loan_kyc_app'])

<br><br><br><hr>

# 14. Credential revocation - Acme revokes Alice's Job-Certificate

In [ ]:
# logger.info("\"Acme\" - Revoke  credential")
acme['alice_cert_rev_reg_delta'] = \
    await anoncreds.issuer_revoke_credential(acme['wallet'],
                                                acme['blob_storage_reader_cfg_handle'],
                                                acme['revoc_reg_id'],
                                                acme['job_certificate_cred_rev_id'])

In [ ]:
# logger.info("\"Acme\" - Post RevocationRegistryDelta to Ledger")
acme['revoc_reg_entry_req'] = \
    await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM',
                                                acme['alice_cert_rev_reg_delta'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_req'])
print()

<br><br><br><hr>

# 15. Apply for the loan with Thrift - Job-Certificate proving (After Revoke)

In [ ]:
# logger.info("\"Thrift\" -> Create \"Loan-Application-Basic\" Proof Request")
nonce = await anoncreds.generate_nonce()
thrift['apply_loan_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Loan-Application-Basic',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {
            'name': 'employee_status',
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        }
    },
    'requested_predicates': {
        'predicate1_referent': {
            'name': 'salary',
            'p_type': '>=',
            'p_value': 2000,
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        },
        'predicate2_referent': {
            'name': 'experience',
            'p_type': '>=',
            'p_value': 1,
            'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
        }
    },
    'non_revoked': {'to': int(time.time())}
})

In [ ]:
# logger.info("\"Thrift\" -> Send \"Loan-Application-Basic\" Proof Request to Alice")
alice['apply_loan_proof_request'] = thrift['apply_loan_proof_request']

In [ ]:
# logger.info("\"Alice\" -> Get credentials for \"Loan-Application-Basic\" Proof Request")
search_for_apply_loan_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['apply_loan_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'attr1_referent')
cred_for_predicate1 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                        'predicate1_referent')
cred_for_predicate2 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                        'predicate2_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_proof_request)

alice['creds_for_apply_loan_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                        cred_for_predicate1['referent']: cred_for_predicate1,
                                        cred_for_predicate2['referent']: cred_for_predicate2}

requested_timestamp = int(json.loads(thrift['apply_loan_proof_request'])['non_revoked']['to'])
alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'], alice['revoc_states_for_loan_app'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                            alice['creds_for_apply_loan_proof'],
                                            alice['name'], None, requested_timestamp)

In [ ]:
# logger.info("\"Alice\" -> Create \"Loan-Application-Basic\" Proof")
revoc_states_for_loan_app = json.loads(alice['revoc_states_for_loan_app'])
timestamp_for_attr1 = get_timestamp_for_attribute(cred_for_attr1, revoc_states_for_loan_app)
timestamp_for_predicate1 = get_timestamp_for_attribute(cred_for_predicate1, revoc_states_for_loan_app)
timestamp_for_predicate2 = get_timestamp_for_attribute(cred_for_predicate2, revoc_states_for_loan_app)
alice['apply_loan_requested_creds'] = json.dumps({
    'self_attested_attributes': {},
    'requested_attributes': {
        'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True,
                            'timestamp': timestamp_for_attr1}
    },
    'requested_predicates': {
        'predicate1_referent': {'cred_id': cred_for_predicate1['referent'],
                                'timestamp': timestamp_for_predicate1},
        'predicate2_referent': {'cred_id': cred_for_predicate2['referent'],
                                'timestamp': timestamp_for_predicate2}
    }
})
alice['apply_loan_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['apply_loan_proof_request'],
                                        alice['apply_loan_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'],
                                        alice['revoc_states_for_loan_app'])

In [ ]:
# logger.info("\"Alice\" -> Send \"Loan-Application-Basic\" Proof to Thrift")
thrift['apply_loan_proof'] = alice['apply_loan_proof']
apply_loan_proof_object = json.loads(thrift['apply_loan_proof'])

In [ ]:
# logger.info("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying")

thrift['schemas_for_loan_app'], thrift['cred_defs_for_loan_app'], thrift['revoc_defs_for_loan_app'], \
thrift['revoc_regs_for_loan_app'] = \
    await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                            apply_loan_proof_object['identifiers'],
                                            thrift['name'], requested_timestamp)

In [ ]:
# logger.info("\"Thrift\" -> Verify \"Loan-Application-Basic\" Proof from Alice")
assert 'Permanent' == apply_loan_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']

assert await anoncreds.verifier_verify_proof(thrift['apply_loan_proof_request'],
                                                thrift['apply_loan_proof'],
                                                thrift['schemas_for_loan_app'],
                                                thrift['cred_defs_for_loan_app'],
                                                thrift['revoc_defs_for_loan_app'],
                                                thrift['revoc_regs_for_loan_app'])

<br><br><br><hr>

# 16. Finish

In [ ]:
# logger.info(" \"Sovrin Steward\" -> Close and Delete wallet")
await wallet.close_wallet(steward['wallet'])
await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

In [ ]:
# logger.info("\"Government\" -> Close and Delete wallet")
await wallet.close_wallet(government['wallet'])
await wallet.delete_wallet(government['wallet_config'],government['wallet_credentials'])

In [ ]:
# logger.info("\"Faber\" -> Close and Delete wallet")
await wallet.close_wallet(faber['wallet'])
await wallet.delete_wallet(faber['wallet_config'], faber['wallet_credentials'])

In [ ]:
# logger.info("\"Acme\" -> Close and Delete wallet")
await wallet.close_wallet(acme['wallet'])
await wallet.delete_wallet(acme['wallet_config'], acme['wallet_credentials'])

In [ ]:
# logger.info("\"Thrift\" -> Close and Delete wallet")
await wallet.close_wallet(thrift['wallet'])
await wallet.delete_wallet(thrift['wallet_config'], thrift['wallet_credentials'])

In [ ]:
# logger.info("\"Alice\" -> Close and Delete wallet")
await wallet.close_wallet(alice['wallet'])
await wallet.delete_wallet(alice['wallet_config'], alice['wallet_credentials'])

In [ ]:
# logger.info("Close and Delete pool")
await pool.close_pool_ledger(pool_['handle'])
await pool.delete_pool_ledger_config(pool_['name'])